## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os 
from dotenv import load_dotenv
#from langchain_openai import ChatOpenAI
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [26]:
from langchain_groq import ChatGroq
model = ChatGroq(
    model = "Gemma2-9b-It",
    groq_api_key = groq_api_key,
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D280543D60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D280677E80>, model_name='Gemma2-9b-It')

In [2]:
model = ChatOpenAI(
    model="gpt-4o-mini",
)

In [3]:
response = model.invoke('What is the capital of France?')
print(response.content)

The capital of France is Paris.


In [16]:
from langchain_core.messages import HumanMessage , SystemMessage

messages = [
    HumanMessage(content="Hi , my name is Adekunle and I am a AI engineer")
]

response = model.invoke(messages)
from IPython.display import display, Markdown

Markdown(response.content)

Hi Adekunle! It's great to meet you. As an AI engineer, you must work on some fascinating projects. What specific areas of AI are you most passionate about?

In [22]:
from langchain_core.messages import AIMessage

result = model.invoke(
    [
        HumanMessage(content="Hi , my name is Adekunle and I am a AI engineer"),
        AIMessage(content= "Hello Adekunle It's nice to meet you. \n\nAs an AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        SystemMessage(content="Hey , what is my name and what do I do?")
    ]
)



In [23]:
result

AIMessage(content='\n', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 88, 'total_tokens': 91, 'completion_time': 0.005454545, 'prompt_time': 0.004106151, 'queue_time': 0.156967386, 'total_time': 0.009560696}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7021a5f-fe0c-4542-83d3-0082fda8f1f5-0', usage_metadata={'input_tokens': 88, 'output_tokens': 3, 'total_tokens': 91})

In [30]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [31]:
store = {}

def get_session_history(session_id : str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model , get_session_history)

In [32]:
config = {'configurable' : {'session_id' : 'chat1'}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi , my name is Adekunle and I am a AI engineer")],
    config = config
)

In [34]:
print(response.content)

Hi Adekunle, it's nice to meet you!

That's awesome that you're an AI engineer. It's a really exciting field right now.  

What kind of AI work are you involved in? 




In [35]:
with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config = config
    
)

AIMessage(content='Your name is Adekunle. 😊  I remember that from our introduction! \n', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 88, 'total_tokens': 108, 'completion_time': 0.036363636, 'prompt_time': 0.004074662, 'queue_time': 0.15603068, 'total_time': 0.040438298}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e19d70cd-fb96-4446-896f-976017e6c2a8-0', usage_metadata={'input_tokens': 88, 'output_tokens': 20, 'total_tokens': 108})

In [36]:
####Change the config---->session_id
config1 = {'configurable' : {'session_id' : 'chat2'}}

response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config = config1
)

print(response.content)

As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know!



In [37]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😊\n"

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config = config1
)

print(response.content)

Your name is John.  

I remember you told me earlier! 😄  



### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [39]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        ('user' ,"{messages}")
    ]
)



In [43]:
chain = prompt | model

response = chain.invoke(
    {"messages" : [HumanMessage(content="Hi , my name is Adekunnle")]} 
)

response.content

"Hello Adekunnle, it's nice to meet you! 👋 \n\nHow can I help you today? 😊 \n"

In [44]:
with_message_history = RunnableWithMessageHistory(chain , get_session_history)

In [47]:
config = {'configurable' : {'session_id' : 'chat3'}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi , my name is Yusuf and I am a AI engineer")],
    config = config
    
)

response

AIMessage(content="Hi Yusuf, it's nice to meet you!  \n\nAs an AI assistant, I'm always happy to help. What can I do for you today?  \n\nAre you working on any interesting projects as an AI engineer?  \n", response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 603, 'total_tokens': 655, 'completion_time': 0.094545455, 'prompt_time': 0.02709599, 'queue_time': 0.15647148, 'total_time': 0.121641445}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2da80d6d-950b-4fea-a91e-2449b0b7bb55-0', usage_metadata={'input_tokens': 603, 'output_tokens': 52, 'total_tokens': 655})

In [48]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config = config
)

response.content

'Your name is Yusuf.  \n\n'

In [57]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages")

        
    ]
)

chain = prompt | model

In [61]:
response = chain.invoke(
    {
        "messages" : [HumanMessage(content="Hi , my name is krish")],
        "language" : "Hindi"
    }
)

response.content

'नमस्ते कृष्ण! मुझे बताओ, मैं आपकी कैसे मदद कर सकता हूँ? 😊\n'

In [62]:
with_message_history = RunnableWithMessageHistory(
    chain ,
    get_session_history,
    input_messages_key= "messages"
)

In [68]:
config = {'configurable' : {'session_id' : 'chat4'}}

response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content="Explain artificial intelligence")],
        "language" : "French"
    },
    config = config
)

response.content

'L\'intelligence artificielle (IA) est un domaine de l\'informatique qui vise à créer des machines capables d\'exécuter des tâches qui nécessitent généralement l\'intelligence humaine. \n\nImaginez un ordinateur qui peut apprendre, résoudre des problèmes, comprendre le langage naturel et même prendre des décisions comme un humain. C\'est l\'objectif de l\'IA. \n\nIl existe plusieurs types d\'IA, allant des systèmes simples qui effectuent des tâches préprogrammées aux systèmes plus avancés capables d\'apprendre à partir de données et de s\'adapter à de nouvelles situations. \n\nVoici quelques exemples concrets d\'applications de l\'IA:\n\n* **Les assistants vocaux** comme Siri ou Alexa utilisent l\'IA pour comprendre vos commandes et y répondre.\n* **Les systèmes de recommandation** sur les plateformes de streaming (Netflix, Spotify) utilisent l\'IA pour vous proposer des films ou des chansons que vous pourriez aimer.\n* **Les voitures autonomes** utilisent l\'IA pour "voir" leur enviro

In [67]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response.content

"Your name is Krish.  😄  It's nice to meet you, Krish! \n\n\n"

In [66]:
response.content

'Votre nom est Krish. 😊 \n'